# **Transformer-Based Chess Engine**

### **References**
**Noever, D., Ciolino, M., & Kalin, J. (2020).**  
*The Chess Transformer: Mastering Play using Generative Language Models.*  
[https://arxiv.org/abs/2008.04057](https://arxiv.org/abs/2008.04057)

In [ ]:
print("Transformer-Based Chess Engine")